In [ ]:
%matplotlib widget

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style

from lvmdrp import log, path, __version__ as drpver
from lvmdrp.utils import metadata as md

from lvmdrp.core.constants import ARC_LAMPS, SPEC_CHANNELS
from lvmdrp.core import rss, image, spectrum1d
from lvmdrp.functions import imageMethod
from lvmdrp.functions import run_drp as drp


style.use(['dark_background', 'ggplot', 'fast'])

In [ ]:
raw_paths = md.locate_new_frames(hemi="s", mjd="6014?", camera="*", expnum="????????")
metadata = md.extract_metadata(raw_paths)

In [ ]:
# adding slitmap
slitmap = drp.read_fibermap(as_table=True)

In [ ]:
# define list of lamps
lamps = [lamp.lower() for lamp in ARC_LAMPS]
# initialize lamps status (OFF)
lamps_status = dict.fromkeys(lamps, False)

arcs = md.get_metadata(tileid="*", mjd="6014?", kind="raw", imagetyp="arc")
arcs

In [ ]:
for lamp in lamps[:-1]:
    # set current lamp status
    lamps_current = lamps_status.copy()
    lamps_current[lamp] = True
    
    # group arcs into analogs of current lamp
    analogs, cpaths, mpaths = md.get_analog_groups(tileid=1111, mjd=60145, imagetyp="arc", **lamps_current)
    
    for i, (keys, analog) in enumerate(analogs.items()):
        # unpack keys
        tileid, mjd, imagetyp, camera = keys
        # match masters
        masters = md.match_master_metadata(target_imagetyp="arc", target_camera=camera, target_mjd=mjd)
        mpixmask = path.full("lvm_master", drpver=drpver, kind="mpixmask", **masters["pixmask"])
        mbias = path.full("lvm_master", drpver=drpver, kind="mbias", **masters["bias"])
        mdark = path.full("lvm_master", drpver=drpver, kind="mdark", **masters["dark"])
        mflat = path.full("lvm_master", drpver=drpver, kind="mflat", **masters["flat"])
        
        # preproc and detrend raw arcs
        for j, (_, arc) in enumerate(analog.iterrows()):
            rpath = path.full("lvm_raw", camspec=arc.camera, **arc.to_dict())
            ppath = path.full("lvm_anc", drpver=drpver, kind="p", imagetype=arc.imagetyp, **arc.to_dict())
            cpath = cpaths[keys][j]
            
            if os.path.isfile(ppath):
                log.info(f"skipping {ppath}, file already exist")
            else:
                os.makedirs(os.path.dirname(ppath), exist_ok=True)
                imageMethod.preproc_raw_frame(in_image=rpath, out_image=ppath, in_mask=mpixmask)
            
            if os.path.isfile(cpath):
                log.info(f"skipping {cpath}, file already exist")
            else:
                imageMethod.detrend_frame(in_image=ppath, out_image=cpath, in_bias=mbias, in_dark=mdark, in_slitmap=slitmap)
        
        # create master arcs
        if os.path.isfile(mpaths[i]):
            log.info(f"skipping {mpath}, file already exist")
        else:
            os.makedirs(os.path.dirname(mpaths[i]), exist_ok=True)
            imageMethod.create_master_frame(in_images=cpaths[keys], out_image=mpaths[i].replace("marc", f"marc_{lamp}"))

In [ ]:
master_paths = [os.path.join(root, file) for root, _, files in os.walk(os.getenv("LVM_SPECTRO_REDUX")) for file in files if file.startswith("lvm-marc_") and file.endswith(".fits")]
master_arcs = md.extract_metadata(kind="master", frames_paths=master_paths)

for _, marc in master_arcs.iterrows():
    lamp = marc[lamps].astype(int).idxmax(0)
    mpath = path.full("lvm_master", drpver=drpver, kind=f"marc_{lamp}", **marc.to_dict())
    xpath = path.full("lvm_master", drpver=drpver, kind=f"xmarc_{lamp}", **marc.to_dict())
    
    mtrace = path.full("lvm_master", drpver=drpver, kind="mtrace", **marc.to_dict())
    mtrace = mtrace.replace("60145", "60115")

    # if os.path.isfile(xpath):
    #     log.info(f"skipping {xpath}, file already exist")
    #     continue
    
    imageMethod.extract_spectra(in_image=mpath, out_rss=xpath, in_trace=mtrace, method="aperture", aperture=3)